In [ ]:
import json
from lxml import html
from lxml.etree import tostring
import os
import shutil
import requests
import re
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
i = 1

home_url = 'https://bokete.jp'
headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
      "referer" : home_url
}

# 保存先のディレクトリを指定
target_dir = "/content/drive/MyDrive/jalan_review/"
if not os.path.exists(target_dir):
        os.mkdir(target_dir)

while 1:
    # カテゴリのトップページ(g2_36がカテゴリによって異なる)
    URL = f'https://www.jalan.net/kankou/g2_36/page_{i}/'

    res = requests.get(URL, headers = headers)
    res.encoding = "shift_jis"
    row_html = html.fromstring(res.text.replace("<br>", "").replace("\t", "").replace("\n", ""))

    # カテゴリを取得
    category = row_html.xpath('//*[@id="contentsListHeader"]/div/h1/text()')[0]
    if not os.path.exists(f"{target_dir}{category}/"):
        os.mkdir(f"{target_dir}{category}/")
    # 既にそのカテゴリについてスクレイピング済みの場合, 続きのページから始める
    if os.path.exists(f"{target_dir}{category}/last_page.json"):
        with open(f"{target_dir}{category}/last_page.json", "r") as f:
            i = json.load(f)["last_page"] + 1
        URL = f'https://www.jalan.net/kankou/g2_36/page_{i}/'

        res = requests.get(URL, headers = headers)
        res.encoding = "shift_jis"
        row_html = html.fromstring(res.text.replace("<br>", "").replace("\t", "").replace("\n", ""))

    # todo 観光地ボックスを指定するようにする
    spt_links = row_html.xpath('//*[@id="cassetteType"]/li[*]/div/div[*]/p[1]/a/@href')
    spt_names = row_html.xpath('//*[@id="cassetteType"]/li[*]/div/div[*]/p[1]/a/text()')

    # 取得できる観光地が無い場合終了
    if len(spt_links) == 0: break

    for SL, SN in zip(spt_links, spt_names):
        SN = SN.replace("/", "_")
        pattern = r"spt_\d+[a-zA-Z]{2,}\d+"
        m = re.search(pattern, SL)
        # パターンと不一致の場合終了
        if not m: continue

        spt_ID = m.group()
        target_path = f"{target_dir}{category}/{spt_ID}_{SN}.json"
        # 既に該当観光地についてスクレイピング済みの場合終了
        if os.path.exists(target_path): continue

        # 観光地のトップページを取得
        URL = f'https://www.jalan.net/kankou/{spt_ID}/'

        res = requests.get(URL, headers = headers)
        res.encoding = "shift_jis"
        row_html = html.fromstring(res.text.replace("<br>", "").replace("\t", "").replace("\n", ""))
        adress = row_html.xpath('//*[@id="detailMap"]/text()')[0]

        # 観光地のレビューを取得
        j = 1
        all_reviews = list()

        while 1:
            URL = f'https://www.jalan.net/kankou/{spt_ID}/kuchikomi/page_{j}'

            res = requests.get(URL, headers = headers)
            res.encoding = "shift_jis"
            row_html = html.fromstring(res.text.replace("<br>", "").replace("\t", "").replace("\n", ""))
            # 全てのレビューボックスを取得
            boxes = row_html.xpath('//*[@id="reviewArea"]/ul/li[*]')

            # 取得できるレビューが無い場合終了
            if len(boxes) == 0: break

            for B in boxes:
                star = float(B.xpath('./div/div[1]/div[1]/span[2]/text()')[0])
                review = B.xpath('./div/div[1]/div[2]/div/span//text()')[-1]
                title = B.xpath('./div/div[1]/p/a/text()')[0]
                tag = B.xpath('./div/div[1]/ul[1]/li/text()')[0]
                auther = B.xpath('./div/div[*]/p[2]/span/text()')[-1]
                posted_date = B.xpath('./div/div[1]/ul[2]/li/text()')[-1][4:]

                all_reviews.append({
                    "title" : title,
                    "review" : review,
                    "posted_date" : posted_date,
                    "star" : star,
                    "tag" : tag,
                    "auther" : auther
                })

            j += 1

            sys.stdout.write('\r')
            sys.stdout.write(f"{SN}, {len(all_reviews)}")
            sys.stdout.flush()

        # jsonファイルに書き込み
        with open(target_path, "w") as f:
            json.dump({
                "name" : SN,
                "spt_ID" : spt_ID,
                "adress" : adress,
                "reviews" : all_reviews
            }, f)

        with open("/content/tmp.json", "w") as f:
            json.dump({
                "name" : SN,
                "spt_ID" : spt_ID,
                "adress" : adress,
                "reviews" : all_reviews
            }, f)

    # カテゴリーの１ページが終了した場合, 終わったページ番号を記録
    with open(f"{target_dir}{category}/last_page.json", "w") as f:
        json.dump({"last_page" : i}, f)

    i += 1

東京スカイツリー, 100

In [ ]:
with open("/content/tmp.json", "r") as f:
    a = json.load(f)

a

{'name': '横浜港シンボルタワー',
 'spt_ID': 'spt_14100ad3350046190',
 'adress': '〒231-0811\u3000神奈川県横浜市中区本牧ふ頭1-16',
 'reviews': [{'title': '旅の終わりももうすぐです',
   'review': ' 横浜港の入口付近にある、灯台のような形をした建物です。船旅をして帰ってくる時には、この建物が旅程の終わりを教えてくれます。',
   'posted_date': '2019年5月14日',
   'star': 5.0,
   'tag': '家族',
   'auther': '男性／40代'},
  {'title': '人が少なくてゆっくりできる',
   'review': '横浜駅からバスで40分程で着きました。バス停の場所を探すのが大変でした。海が近いということもあり、風が強かったです。タワー最上階の望遠鏡？が全て故障中でした。近くの休憩所には軽食自販機(全て370円)やアイスの自販機もありました。お土産も売っているようです。ポルノグラフィティのMV撮影地ということで行き、昔はファン用のノートがあったようですが今はありませんでした。そこが残念です。帰りのバスは1時間に1本でした。',
   'posted_date': '2019年3月9日',
   'star': 5.0,
   'tag': '友達同士',
   'auther': '女性／20代'},
  {'title': '最高の眺め',
   'review': 'ここから見る海の景色は本当に最高。人も少なくて爽快な気分になれる公園だ。ただし非常に不便な場所にある。そのため人が少ないとも言える。車でないと行くのが難しいような場所だ。',
   'posted_date': '2018年9月24日',
   'star': 5.0,
   'tag': 'カップル・夫婦',
   'auther': '男性／40代'},
  {'title': 'デートにオススメ',
   'review': '公園のすぐ隣にあり、景色がとても綺麗でした。混んでいないところも良かったです。デートにオススメです。',
   'posted_date': '2018年3月23日',
   'sta